In [1]:
from utils import process_ns3_csv
filename = '20241119_st-100_N-100_BH-40_SF-0_L-20_uni-1'
input_file = './dataset/' + filename + '/MonitorSnifferRx.csv'

output_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx.csv'
log_file = './dataset/' + filename + '/log1119_BH0.4_uniform_20#%20seed_12345.txt'
process_ns3_csv(input_file, output_file, log_file)

In [2]:
from utils import load_data
import numpy as np
import models 

In [3]:
data_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx.csv'

In [4]:
np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
processed_data, adj, T, group_num = load_data(1,data_file)
# print(processed_data[0].edge_attr)
edge_feature_size = processed_data[0].edge_attr.shape[1]
nodes_feature_size = processed_data[0].x.shape[1]


D:\zhuomian\file\弱网中鲁棒性的恶意节点检测\exp\utils.py:547: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:653.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [5]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x.shape）
print(processed_data[0].x)
# print(adj[12])
# print(T[12])

tensor([[0.9869, 0.5195, 0.9909],
        [0.9869, 0.5688, 0.9909],
        [0.9960, 0.6130, 0.9973],
        ...,
        [0.9869, 0.4909, 0.9909],
        [0.9869, 0.5065, 0.9909],
        [0.9869, 0.5870, 0.9909]])


In [6]:
# 打印边连接关系（processed_data.edge_index）
# print(processed_data[-1].edge_index.numpy().size)

In [7]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].y.numpy())
# print(processed_data[0].x[:, 0:23].numpy())
# print(processed_data[15].y.numpy())
# print(processed_data[15].edge_index)
# print(processed_data[12].edge_weights)
# print(processed_data[12].x)

In [8]:
train_model = models.HypergraphModel(nodes_feature_size, edge_feature_size)
print(train_model)

HypergraphModel(
  (layer1): HypergraphConvLayer(
    (conv): HypergraphConv(3, 8)
  )
  (edge_conv): GraphConvolution()
  (class_classifier): Sequential(
    (0): Linear(in_features=8, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=2, bias=True)
  )
  (norm1): CustomBatchNorm()
  (norm2): CustomBatchNorm()
)


In [9]:
import time
import torch.optim as optim
from utils import accuracy, auc, precision, recall, f1_score
import torch.nn.functional as F
import torch
criteria = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
train_end_index = 4
val_index = 6
def train(epoch, save_path):
    t = time.time()
    train_model.train()
    optimizer.zero_grad()
    # print(processed_data[0].x[0:23].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(train_end_index):
        print(i)
        output = train_model(processed_data[i].x, processed_data[i].edge_index, 
                                 processed_data[i].edge_weights, processed_data[i].edge_attr, adj[i], T[i])
        # print(output)
        labels = processed_data[i].y
        loss_train = criteria(output, labels)
        # print(output[:, 1])
        acc_train = acc_measure(output, labels)
        
        loss_train.backward(retain_graph=True)
        optimizer.step()

        
        train_model.eval()
        output = train_model(processed_data[val_index].x, processed_data[val_index].edge_index, 
                                 processed_data[val_index].edge_weights, processed_data[val_index].edge_attr, adj[val_index], T[val_index])
        # print(output)
        labels = processed_data[val_index].y
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        auc_val = auc(output, labels)
        per_val = precision(output, labels)
        recall_val = recall(output, labels)
        f1_val = f1_score(output, labels)
        print(
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
            'auc_val: {:.4f}'.format(auc_val),
            'per_val: {:.4f}'.format(per_val),
            'recall_val: {:.4f}'.format(recall_val),
            'f1_val: {:.4f}'.format(f1_val),
              'time: {:.4f}s'.format(time.time() - t))
    # 每个 epoch 结束后保存模型
    torch.save(train_model.state_dict(), save_path)
    print(f'Model saved to {save_path}')

    return loss_val.item()


In [10]:
def test():
    train_model.eval()
    testIndex = 8
    output = train_model(processed_data[testIndex].x, processed_data[testIndex].edge_index, 
                             processed_data[testIndex].edge_weights, processed_data[testIndex].edge_attr, adj[testIndex], T[testIndex])
    labels = processed_data[group_num-1].y
    # print(output)
    # print(labels)
    loss_test = criteria(output, labels)
    acc_test = acc_measure(output, labels)
    auc_test = auc(output, labels)
    per_test = precision(output, labels)
    recall_test = recall(output, labels)
    f1_test = f1_score(output, labels)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
         'auc_test: {:.4f}'.format(auc_test),
            'per_test: {:.4f}'.format(per_test),
            'recall_test: {:.4f}'.format(recall_test),
            'f1_test: {:.4f}'.format(f1_test))
    return acc_test.item()

In [ ]:
import os
# torch.autograd.set_detect_anomaly(True)
num_epochs = 1000
early_stopping = 30
val_watch = []
t_total = time.time()
current_time = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
save_path = './models/' + current_time + '/' + current_time + '.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)  # 创建目录，如果目录已存在不抛出错误
for epoch in range(num_epochs):
    val = train(epoch, save_path)  # 训练并获得结果
    if val is None:  # 检查是否返回了None值
        print(f"Warning: train(epoch) returned None for epoch {epoch}. Skipping this iteration.")
        continue
    
    val_watch.append(val)  # 将结果添加到val_watch
    test()
    # if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
    #     print("Early stopping...")
    #     break
    
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Printing the weights : ")

0
loss_train: 0.7019 acc_train: 0.3939 loss_val: 3.0570 acc_val: 0.4040 auc_val: 0.4208 per_val: 0.4040 recall_val: 1.0000 f1_val: 0.5755 time: 0.0307s
1
loss_train: 3.3616 acc_train: 0.4040 loss_val: 2.8007 acc_val: 0.4040 auc_val: 0.4203 per_val: 0.4040 recall_val: 1.0000 f1_val: 0.5755 time: 0.0560s
2
loss_train: 3.0276 acc_train: 0.4040 loss_val: 2.5674 acc_val: 0.4040 auc_val: 0.4186 per_val: 0.4040 recall_val: 1.0000 f1_val: 0.5755 time: 0.0818s
3
loss_train: 2.6389 acc_train: 0.4040 loss_val: 2.3643 acc_val: 0.4040 auc_val: 0.4225 per_val: 0.4040 recall_val: 1.0000 f1_val: 0.5755 time: 0.1077s
Model saved to ./models/2024-12-06_16-37-57/2024-12-06_16-37-57.pth
Test set results: loss= 3.7643 accuracy= 0.4040 auc_test: 0.4644 per_test: 0.4040 recall_test: 1.0000 f1_test: 0.5755
0
loss_train: 0.6996 acc_train: 0.3939 loss_val: 1.3592 acc_val: 0.4040 auc_val: 0.4081 per_val: 0.4040 recall_val: 1.0000 f1_val: 0.5755 time: 0.0247s
1
loss_train: 1.4810 acc_train: 0.4040 loss_val: 1.285

In [18]:
# 加载模型

train_model.load_state_dict(torch.load(save_path, weights_only=True))
train_model.eval()  # 切换到评估模式

FileNotFoundError: [Errno 2] No such file or directory: './models/2024-12-06_16-09-04/2024-12-06_16-09-04.pth'